In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

from pandas.plotting import scatter_matrix

from package import encoder

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

Loaded  7 encoders.


In [2]:
description = pd.read_csv("data/dicionario.csv", encoding='latin-1')
train = pd.read_csv("data/dados_treino_hackaton.csv", encoding='latin-1', index_col=False)
test = pd.read_csv("data/dados_teste_x_hackaton.csv", encoding='latin-1', index_col=False)

del train['Unnamed: 0']
del test['Unnamed: 0']

In [3]:
train = encoder.encode_DataFrame(train)
test = encoder.encode_DataFrame(test)

## testing columns correlations

In [5]:
correlation = train.corr()
correlation["id_fechou"].sort_values(ascending=False)[0:20]

id_fechou                             1.000000
Classificacao_da_Oportunidade         0.264357
numero_relacionamentos_convertidos    0.220502
Forma_de_Contratacao_Agregado         0.190301
numero_relacionamentos                0.158940
Data_de_criacao                       0.145784
Gestão de Gastos                      0.135981
Codigo_da_oportunidade                0.128357
Desdobramento de metas                0.127610
Produtos digitais                     0.120585
Gestão da Segurança Viária            0.105962
Gestão de operações projetizadas      0.094878
Gestão da Receita                     0.088548
Software                              0.084177
Skill_dev                             0.062083
Comissão sobre Parceiros              0.060302
Gestão da Saúde                       0.043082
Equilíbrio fiscal                     0.027894
Gestão da Educação                    0.022237
Total_HH                              0.022200
Name: id_fechou, dtype: float64

Columns that dont have any data on it should be removed

In [5]:
print(train['Gestão da Segurança Pública'].value_counts())
print(train['S_amp_OP_S_amp_OE'].value_counts())
print(train['Transformação Digital'].value_counts())
print(train['Roadmap'].value_counts())

0    613
Name: Gestão da Segurança Pública, dtype: int64
0    613
Name: S_amp_OP_S_amp_OE, dtype: int64
0    613
Name: Transformação Digital, dtype: int64
0    613
Name: Roadmap, dtype: int64


#### Defining columns to remove

In [6]:
# Removendo datas ou codigos unicos de clientes. 
remove_columns = ['Data_de_criacao', 'ano', 'ID_cliente', 'Codigo_da_oportunidade', 'Gestão da Segurança Pública', 
                 'S_amp_OP_S_amp_OE', 'Transformação Digital', 'Roadmap']

#### Creating new columns that may have a impact

In [7]:
train["Custo_Total_per_Valor_corrigido2"] = train["Custo_Total"]/train["Valor_corrigido2"]
train["numero_relacionamentos_convertidos_per_numero_relacionamentos"] = train["numero_relacionamentos_convertidos"]/train["numero_relacionamentos"]
train["Gestão da Receita_per_Gestão de Gastos"] = train["Gestão da Receita"] + train["Gestão de Gastos"]

## new columns correlations

New columns have a bigger correlation

In [8]:
correlation = train.corr()
correlation["id_fechou"].sort_values(ascending=False)[0:20]

id_fechou                                                        1.000000
Classificacao_da_Oportunidade                                    0.264357
numero_relacionamentos_convertidos_per_numero_relacionamentos    0.258246
numero_relacionamentos_convertidos                               0.220502
Forma_de_Contratacao_Agregado                                    0.190301
Gestão da Receita_per_Gestão de Gastos                           0.165818
numero_relacionamentos                                           0.158940
Data_de_criacao                                                  0.145784
Gestão de Gastos                                                 0.135981
Codigo_da_oportunidade                                           0.128357
Desdobramento de metas                                           0.127610
Produtos digitais                                                0.120585
Custo_Total_per_Valor_corrigido2                                 0.119211
Gestão da Segurança Viária            

### its very important to scale columns, ML algorithms usually do not work well with large range data

In [249]:
scaler = StandardScaler()

In [250]:
print(scaler.fit(X_train))

StandardScaler()


In [251]:
scaler.transform(X_train)

array([[-0.02226183,  0.5155858 ,  0.42184818, ..., -0.60345916,
        -0.92295326, -0.30618622],
       [-1.1130917 , -0.45237221,  0.28111898, ...,  0.56929319,
        -0.92295326, -0.30618622],
       [-1.1130917 ,  1.4835438 , -1.02382451, ...,  0.62770536,
        -0.92295326, -0.30618622],
       ...,
       [-1.1130917 ,  1.4835438 , -0.16665575, ...,  1.09188611,
        -0.92295326, -0.30618622],
       [ 1.06856803, -1.42033022,  0.7288937 , ...,  0.34607502,
         0.57819907, -0.30618622],
       [-1.1130917 ,  0.9995648 , -1.19014084, ..., -1.00780678,
        -0.92295326, -0.30618622]])

In [9]:
train = train.drop(remove_columns, axis=1)

In [10]:
train_labels, target_label = generate_labels(train)

X_train, X_test, y_train, y_test = train_test_split(train[train_labels], train[target_label], test_size=0.2, random_state=42)

In [20]:
from package.models import run_XGB, run_analysis

In [13]:
run_XGB(X_train, X_test, y_train, y_test, 3, 2.7, 1.3, 0.1, 0.3)

(XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False, eta=1.3,
               eval_metric=None, gamma=2.7, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=1.29999995, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=100,
               n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
               reg_alpha=0.1, ...),
 0.6910569105691057)

In [253]:
lm = 0
for md in np.arange(1, 5):
    for gam in np.arange(0.1, 3, 0.2):
        for eta in np.arange(0.1, 2, 0.2):
            for ra in np.arange(0.1, 1.5, 0.2):
                for rl in np.arange(0.1, 1.5, 0.2):
                    _, r = run_XGB(scaler.transform(X_train), scaler.transform(X_test), y_train, y_test, md, gam, eta, ra, rl)
                    if r > lm:
                        lm = r
                        print("HIGHER ", lm)
                        print(md, gam, eta, ra, rl)
                        print("")

HIGHER  0.7317073170731707
1 0.1 0.1 0.1 0.1

HIGHER  0.7723577235772358
1 0.1 0.30000000000000004 0.1 0.1

HIGHER  0.7804878048780488
1 0.1 0.30000000000000004 1.1000000000000003 1.3000000000000003

HIGHER  0.7886178861788617
1 0.7000000000000001 0.30000000000000004 0.9000000000000001 1.3000000000000003

HIGHER  0.8048780487804879
1 1.1000000000000003 1.7000000000000004 1.1000000000000003 0.5000000000000001



KeyboardInterrupt: 

In [12]:
from sklearn.metrics import accuracy_score, classification_report

In [107]:
run_XGB(2, 0.3, 0.5, 0.5, 0.7)

              precision    recall  f1-score   support

           0       0.84      0.78      0.81        68
           1       0.75      0.82      0.78        55

    accuracy                           0.80       123
   macro avg       0.80      0.80      0.80       123
weighted avg       0.80      0.80      0.80       123



0.7967479674796748

In [254]:
run_XGB(scaler.transform(X_train), scaler.transform(X_test), y_train, y_test, 2, 0.1, 1.5, 1.1, 0.7, prints=True)

Running XGB model.
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        63
           1       0.82      0.82      0.82        60

    accuracy                           0.82       123
   macro avg       0.82      0.82      0.82       123
weighted avg       0.82      0.82      0.82       123

accuracy_score:  0.8211382113821138





(XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False, eta=1.5,
               eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=1.5, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=100,
               n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
               reg_alpha=1.1, ...),
 0.8211382113821138)

In [688]:
from pickle import dump

In [16]:
# Stochastic Gradient Descent

In [13]:
from sklearn.linear_model import SGDClassifier, SGDRegressor

In [14]:
clf = SGDRegressor(loss="epsilon_insensitive", penalty="l2", max_iter=500)
clf.fit(X_train, y_train)

SGDRegressor(loss='epsilon_insensitive', max_iter=500)

In [15]:
from lightgbm import LGBMClassifier

In [68]:
mo = LGBMClassifier(learning_rate=0.1)

In [69]:
mo.fit(X_train, y_train)

LGBMClassifier(n_estimators=200)

In [70]:
run_analysis(mo.predict(X_test), y_test)

              precision    recall  f1-score   support

           0       0.75      0.71      0.73        66
           1       0.68      0.72      0.70        57

    accuracy                           0.72       123
   macro avg       0.71      0.72      0.71       123
weighted avg       0.72      0.72      0.72       123

accuracy_score:  0.7154471544715447





In [677]:
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense, Dropout       # importing Dense layers

In [678]:
model = Sequential()

In [679]:
model.add(Dense(64, activation='relu', input_shape=(38,)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [680]:
lr_schedule = keras.optimizers.schedules.learning_rate_schedule.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)


opt = keras.optimizers.RMSprop(learning_rate=lr_schedule)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [683]:
hist = model.fit(scaler.transform(X_train), y_train, validation_data=(scaler.transform(X_test), y_test), epochs=40, verbose=1)

Epoch 1/40
16/16 [==============================] - 0s 5ms/step - loss: 0.5606 - accuracy: 0.7510 - val_loss: 0.6748 - val_accuracy: 0.6829
Epoch 2/40
16/16 [==============================] - 0s 4ms/step - loss: 0.5405 - accuracy: 0.7571 - val_loss: 0.6725 - val_accuracy: 0.6829
Epoch 3/40
16/16 [==============================] - 0s 3ms/step - loss: 0.5610 - accuracy: 0.7449 - val_loss: 0.6756 - val_accuracy: 0.6748
Epoch 4/40
16/16 [==============================] - 0s 4ms/step - loss: 0.5529 - accuracy: 0.7388 - val_loss: 0.6756 - val_accuracy: 0.6748
Epoch 5/40
16/16 [==============================] - 0s 4ms/step - loss: 0.5420 - accuracy: 0.7429 - val_loss: 0.6759 - val_accuracy: 0.6829
Epoch 6/40
16/16 [==============================] - 0s 4ms/step - loss: 0.5329 - accuracy: 0.7469 - val_loss: 0.6685 - val_accuracy: 0.6748
Epoch 7/40
16/16 [==============================] - 0s 4ms/step - loss: 0.5398 - accuracy: 0.7429 - val_loss: 0.6710 - val_accuracy: 0.6829
Epoch 8/40
16/16 [==